# Spacy
[Curso Spacy](https://course.spacy.io/es/)

In [ ]:
# Importamos la librería spacy
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Encontrando palabras, frases, nombres y conceptos

### Objetos

#### NLP

In [ ]:
# Creamos un objeto nlp vacío para procesar español
nlp = spacy.blank("es")

#### DOC

In [ ]:
# Creamos un doc para procesar un string de texto con el objeto NLP
doc1 = nlp("Hola, mi nombre es Javier y tengo 20 años")

# Mostramos todos los tokens línea a línea
for token in doc1:
    print(token.text)

Hola
,
mi
nombre
es
Javier
y
tengo
20
años


In [ ]:
# Insertamos el tercer token en una variable
token = doc1[2]

# Imprimimos la variable con el atributo .text
print(token.text)

mi


#### SPAN

In [ ]:
# Obtenemos del tercer al sexto token
span = doc1[2:6]

# Imprimimos la variable gracias al atributo .text
print(span.text)

mi nombre es Javier


### Atributos léxicos

In [ ]:
# Imprime todas las posiciones disponibles
print("Total posiciones: ", [token.i for token in doc1])

Posiciones:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# Imprime todo el texto
print("Total texto: ", [token.text for token in doc1])

Texto:  ['Hola', ',', 'mi', 'nombre', 'es', 'Javier', 'y', 'tengo', '20', 'años']


In [ ]:
# Devuelve True o False si los tokens son alfanuméricos o no
print("Alfanuméricos:", [token.is_alpha for token in doc1])

Alfanuméricos: [True, False, True, True, True, True, True, True, False, True]


In [ ]:
# Devuelve True o False si los tokens son signos de puntuación o no
print("Signos de puntuación:", [token.is_punct for token in doc1])

Signos de puntuación: [False, True, False, False, False, False, False, False, False, False]


In [ ]:
# Devuelve True o False si los tokens son números o no
print("Números:", [token.like_num for token in doc1])

Números: [False, False, False, False, False, False, False, False, True, False]


### Pipelines entrenados

In [ ]:
# Descargamos este pipeline preentrenado
! python -m spacy download es_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-07 17:27:24.623548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 17:27:24.623653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 17:27:24.623672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
# Cargamos el pipeline en un objeto nlp
nlp = spacy.load("es_core_news_sm")

Atributos:
* .text (Imprime el texto)
* .pos_ (Imprime la etiqueta gramatical)
* .dep_ (Imprime la etiqueta de la dependencia sintáctica)
* .head.text (Devuelve el token "padre" de la palabra)

In [ ]:
doc2 = nlp("Mi casa es azul")

for token in doc2:
    print(token.text, token.pos_, token.dep_, token.head.text)

Mi DET nsubj azul
casa NOUN flat Mi
es AUX cop azul
azul ADJ ROOT azul


In [ ]:
# Ingresamos un texto con entidades
doc3 = nlp("Seat es una marca de coches de Alemania que proviene de Volkswagen")

for entidad in doc3.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(entidad.text, entidad.label_)

Seat ORG
Alemania LOC
Volkswagen ORG


### Patrones basados en reglas

#### Casos de uso

In [ ]:
# Aquí estamos buscando tres tokens con el texto "camiseta", "blanca" y "rayas".
[{"TEXT": "camiseta"}, {"TEXT": "blanca"}, {"TEXT": "rayas"}]

[{'TEXT': 'camiseta'}, {'TEXT': 'blanca'}, {'TEXT': 'rayas'}]

In [ ]:
# Aquí estamos buscando dos tokens que en minúsculas sean "camiseta" y "blanca".
[{"LOWER": "camiseta"}, {"LOWER": "blanca"}]

[{'LOWER': 'camiseta'}, {'LOWER': 'blanca'}]

In [ ]:
# Aquí estamos buscando cualquier forma del verbo comprar
# seguida de un sustantivo y un adjetivo,
# como por ejemplo: "comprando camisetas blancas"
[{"LEMMA": "comprar"}, {"POS": "NOUN"}, {"POS": "ADJ"}]

[{'LEMMA': 'comprar'}, {'POS': 'NOUN'}, {'POS': 'ADJ'}]

#### Usando el matcher

In [ ]:
# Importamos el matcher
from spacy.matcher import Matcher

In [ ]:
# Iniciamos el matcher con un vocabulario compartido
matcher = Matcher(nlp.vocab)

##### Ejemplo 1

In [ ]:
# Añadimos el patrón al matcher
patron1 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("CAMISETAS", [patron1])

In [ ]:
doc4 = nlp("El otro día mi madre me compró una camiseta blanca en aquella tienda")

In [ ]:
# LLamamos al matcher sobre el doc creado anteriormente
matches = matcher(doc4)

In [ ]:
for match_id, start, end in matches:
    # Obtenemos los resultados
    matched_span = doc4[start:end]
    print(matched_span.text)

camiseta blanca


##### Ejemplo 2

In [ ]:
# Creamos el patrón
patron2 = [
    {'IS_PUNCT': True},
    {'LOWER': 'españa'},
    {'LEMMA': 'ganar'},
    {'LOWER': 'el'},
    {'LOWER': 'mundial'},
    {'LOWER': 'en'},
    {'IS_DIGIT': True},
    {'IS_PUNCT': True}]

In [ ]:
# Lo añadimos al matcher
matcher.add("FUTBOL", [patron2])

In [ ]:
doc5 = nlp("¡España ganó el mundial en 2010! Qué recuerdos")

In [ ]:
matches = matcher(doc5)

In [ ]:
# Imprimimos los resultados que coincidan
for match_id, start, end in matches:
    matched_span = doc5[start:end]
    print(matched_span.text)

¡España ganó el mundial en 2010!


##### Ejemplo 3

In [ ]:
# Creamos un patron
patron3 = [
    {"LEMMA": "comprar"},
    {"POS": "DET", "OP": "?"},  # opcional: encuentra 0 o 1 ocurrencias
    {"POS": "NOUN"}
]

In [ ]:
# Lo añadimos al matcher
matcher.add("COCHES", [patron3])

In [ ]:
doc6 = nlp('La semana pasada me compré el coche de mis sueños, ayer le a compré cosas para decorar')

In [ ]:
matches = matcher(doc6)

In [ ]:
# Imprimimos los resultados que coincidan
for match_id, start, end in matches:
    matched_span = doc6[start:end]
    print(matched_span.text)

semana pasada
compré el coche
compré cosas


## Análisis de datos a gran escala

### Estructuras de datos

#### Vocabulario compartido

In [ ]:
# Añadimos la palabra coche
nlp.vocab.strings.add("coche")

# Añadimos el hash de la palabra coche
coche_hash = nlp.vocab.strings["coche"]

# Mostramos el has
coche_string = nlp.vocab.strings[coche_hash]

# Mostramos el string y su hash
print(coche_string, ':', coche_hash)

coche : 6709559885411883416


In [ ]:
# Podemos obtener el string a través del hash
string = nlp.vocab.strings[6709559885411883416]

print(string)

coche


In [ ]:
# Podemos buscar el string / hash a través de un doc
doc7 = nlp("Me he comprado un coche nuevo")

print("Hash:", nlp.vocab.strings["coche"])
print("String:", nlp.vocab.strings[6709559885411883416])

Hash: 6709559885411883416
String: coche


#### Lexemas

In [ ]:
lexema = nlp.vocab["coche"]

# Imprime en pantalla los atributos léxicos
print(lexema.text, lexema.orth, lexema.is_alpha)

coche 6709559885411883416 True


#### Doc

In [ ]:
# Importamos la clase Doc
from spacy.tokens import Doc

In [ ]:
# Las palabras y espacios que usaremos para crear el doc
palabras = ["¡" , "Me", "llamo", "Javier", "!"]
espacios = [False, True, True, False, True]

# Crea un doc manualmente
doc8 = Doc(nlp.vocab, words=palabras, spaces=espacios)

In [ ]:
print(doc8)

¡Me llamo Javier! 


#### Span

In [ ]:
# Importamos la clase Span
from spacy.tokens import Span

In [ ]:
# Crea un span manualmente
span = Span(doc8, 1, 3)

# Crea un span con un label
span_with_label = Span(doc8, 1, 3, label="SALUDO")

# Añade el span a los doc.ents
doc8.ents = [span_with_label]

In [ ]:
span

Me llamo

### Word vectors y similitud semántica

In [ ]:
!python -m spacy download es_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-07 17:36:20.157300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 17:36:20.157436: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 17:36:20.157460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
# Cargamos un modelo más grande
nlp = spacy.load("es_core_news_md")

In [ ]:
# Creamos 2 docs y los comparamos
doc9 = nlp("Me he comprado una camiseta verde")
doc10 = nlp("Me compré una camiseta blanca")
print(doc9.similarity(doc10))

0.7307724459016532


In [ ]:
# Comparamos dos tokens
doc11 = nlp("Me gustan los coches y las motos")
token1 = doc11[3]
token2 = doc11[6]
print(token1.similarity(token2))

0.7295453548431396


In [ ]:
# Comparamos un doc con un token
doc12 = nlp("Me gustan los coches")
token = nlp("camiseta")[0]

print(doc12.similarity(token))

0.12287714437993383


In [ ]:
# Comparamos un span con un documento
span = nlp("Me gustan los coches rojos")[3:5]
doc13 = nlp("Ferrari hace coches")

print(span.similarity(doc13))

0.534885437177009


### Combinando modelos y reglas

In [ ]:
# Importamos el PhraseMatcher
from spacy.matcher import PhraseMatcher

In [ ]:
# Iniciamos el matcher con un vocabulario compartido
matcher = PhraseMatcher(nlp.vocab)

In [ ]:
# Creamos el patron que queremos buscar
patron = nlp("coche rojo")

# Añadimos el patron al matcher
matcher.add("COCHE", [patron])

# Creamos un doc
doc14 = nlp("Me compré el otro dia mi primer coche rojo")

In [ ]:
# Imprimimos los resultados
for match_id, start, end in matcher(doc14):
    span = doc14[start:end]
    print(span.text)

coche rojo
